# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import DAGMM as Model
from pytorch_utils import filtered_load_model
import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
#with open("./config.yaml", 'rb') as f:
#    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/research/dcase2020/result/2D/DAGMM/meta_dense/latent5_mixture3_revised_euclid_no3/config.yaml'

## make path list and train/valid split

In [7]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

In [8]:
for machine_type in machine_types:
    # dev train
    dev_train_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths = sorted(dev_train_paths)
    # add_dev train
    add_train_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_paths = sorted(add_train_paths)
    # valid
    dev_valid_paths = ["{}/{}/test/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    dev_valid_paths = sorted(dev_valid_paths)
    
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths + add_train_paths
    train_paths[machine_type]['valid'] = dev_valid_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(train_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)         # debug
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['preprocessing']['sample_rate'],
                window_size=config['preprocessing']['window_size'],
                hop_size=config['preprocessing']['hop_size'],
                mel_bins=config['preprocessing']['mel_bins'],
                fmin=config['preprocessing']['fmin'],
                fmax=config['preprocessing']['fmax'],
                latent_size=config['fit']['latent_size'],
                mixture_size=config['fit']['mixture_size'])
    #pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_PATH'])
    #net = filtered_load_model(net, pretrained_dict)
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['fit']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e2, 
                                              max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    criterion = nn.MSELoss()
    #try:
    history = modeler.train_net(net, dataloaders_dict, criterion, optimizer, scheduler, num_epochs, writer)
    # output
    model = history['model']
    gmm_param = history['gmm_param']
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    gmm_param_path = MODEL_DIR+'/{}_gmm_param.pkl'.format(machine_type)
    torch.save(model.state_dict(), model_out_path)
    logger.info('\n success:{0} \n'.format(machine_type) + \
                    'model_out_path ==> \n {0}'.format(model_out_path))
    pd.to_pickle(gmm_param, gmm_param_path)
    #  close writer for tensorbord
    writer.close()
    #modeler.mlflow_log(history, config, machine_type, model_out_path, tb_log_dir)
    com.toc()
    #except:
    #    print("error")
    #    history = "error"
    return history

In [10]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

In [ ]:
for machine_type in machine_types:
    history = run(machine_type)
    #with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    #    pickle.dump(history , file)

2020-12-01 00:35:15,028 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2020-12-01 00:35:15,029 - 00_train.py - INFO - MAKE DATA_LOADER
2020-12-01 00:35:15,851 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|██████████| 29/29 [00:02<00:00, 10.69it/s]
2020-12-01 00:35:38,585 - pytorch_modeler.py - INFO - Epoch 1/300:train_loss:96314.596375, tr_rec:96236.223450, tr_eng:563.208105, tr_covd:4410.438313, val_AUC:0.479387, val_pAUC:0.490584
100%|██████████| 29/29 [00:02<00:00, 10.41it/s]
2020-12-01 00:36:05,758 - pytorch_modeler.py - INFO - Epoch 2/300:train_loss:95172.680481, tr_rec:95112.717651, tr_eng:404.236800, tr_covd:3907.836300, val_AUC:0.507937, val_pAUC:0.510523
100%|██████████| 29/29 [00:02<00:00, 10.69it/s]
2020-12-01 00:36:33,217 - pytorch_modeler.py - INFO - Epoch 3/300:train_loss:94208.542175, tr_rec:94162.635681, tr_eng:276.425233, tr_covd:3652.802299, val_AUC:0.509100, val_pAUC:0.492075
100%|██████████| 29/29 [00:02<00:00, 10.43it/s]
2020-12-01 00:36:59,203 - pytorch_modeler.py - INFO - Epoch 4/300:train_loss:93151.115784, tr_rec:93115.108337, tr_eng:184.195693, tr_covd:3517.591835, val_AUC:0.533323, val_pAUC:0.516909
100%|██████████| 29/29 [00:02<00:00, 10.50it/s]
2020-12-

100%|██████████| 29/29 [00:01<00:00, 19.28it/s]
2020-12-01 00:44:42,204 - pytorch_modeler.py - INFO - Epoch 30/300:train_loss:615.939665, tr_rec:628.633934, tr_eng:-231.881779, tr_covd:2098.781759, val_AUC:0.483926, val_pAUC:0.491794
100%|██████████| 29/29 [00:01<00:00, 15.47it/s]
2020-12-01 00:44:59,025 - pytorch_modeler.py - INFO - Epoch 31/300:train_loss:613.388053, tr_rec:624.283121, tr_eng:-214.210090, tr_covd:2105.188284, val_AUC:0.457450, val_pAUC:0.495395
100%|██████████| 29/29 [00:01<00:00, 19.54it/s]
2020-12-01 00:45:14,711 - pytorch_modeler.py - INFO - Epoch 32/300:train_loss:596.988135, tr_rec:620.731925, tr_eng:-339.573024, tr_covd:2042.702230, val_AUC:0.454240, val_pAUC:0.497257
100%|██████████| 29/29 [00:01<00:00, 19.04it/s]
2020-12-01 00:45:30,469 - pytorch_modeler.py - INFO - Epoch 33/300:train_loss:585.475600, tr_rec:613.809750, tr_eng:-385.895217, tr_covd:2051.075356, val_AUC:0.493448, val_pAUC:0.500830
100%|██████████| 29/29 [00:01<00:00, 18.84it/s]
2020-12-01 00:45

100%|██████████| 29/29 [00:01<00:00, 19.78it/s]
2020-12-01 00:53:55,179 - pytorch_modeler.py - INFO - Epoch 59/300:train_loss:542.774869, tr_rec:558.226658, tr_eng:-256.185036, tr_covd:2033.342682, val_AUC:0.464216, val_pAUC:0.491241
100%|██████████| 29/29 [00:01<00:00, 18.99it/s]
2020-12-01 00:54:10,908 - pytorch_modeler.py - INFO - Epoch 60/300:train_loss:531.242379, tr_rec:558.754701, tr_eng:-376.751105, tr_covd:2032.556692, val_AUC:0.432486, val_pAUC:0.491728
100%|██████████| 29/29 [00:01<00:00, 19.16it/s]
2020-12-01 00:54:27,265 - pytorch_modeler.py - INFO - Epoch 61/300:train_loss:535.079255, tr_rec:555.873265, tr_eng:-309.026471, tr_covd:2021.727480, val_AUC:0.446663, val_pAUC:0.507235
100%|██████████| 29/29 [00:01<00:00, 18.24it/s]
2020-12-01 00:54:43,465 - pytorch_modeler.py - INFO - Epoch 62/300:train_loss:537.339125, tr_rec:552.506739, tr_eng:-252.023486, tr_covd:2006.947424, val_AUC:0.450273, val_pAUC:0.491015
100%|██████████| 29/29 [00:01<00:00, 18.61it/s]
2020-12-01 00:54

...

100%|██████████| 20/20 [00:02<00:00,  7.21it/s]
2020-12-01 09:31:46,762 - pytorch_modeler.py - INFO - Epoch 263/300:train_loss:341.647409, tr_rec:337.546125, tr_eng:-29.996487, tr_covd:1420.186689, val_AUC:0.773551, val_pAUC:0.514046
100%|██████████| 20/20 [00:03<00:00,  5.46it/s]
2020-12-01 09:32:24,883 - pytorch_modeler.py - INFO - Epoch 264/300:train_loss:337.374644, tr_rec:333.895197, tr_eng:-35.637982, tr_covd:1408.648659, val_AUC:0.794801, val_pAUC:0.564460
100%|██████████| 20/20 [00:03<00:00,  5.42it/s]
2020-12-01 09:33:01,719 - pytorch_modeler.py - INFO - Epoch 265/300:train_loss:342.788107, tr_rec:337.152407, tr_eng:-14.270004, tr_covd:1412.540329, val_AUC:0.792160, val_pAUC:0.535337
100%|██████████| 20/20 [00:03<00:00,  5.49it/s]
2020-12-01 09:33:39,163 - pytorch_modeler.py - INFO - Epoch 266/300:train_loss:339.366497, tr_rec:334.495405, tr_eng:-22.085506, tr_covd:1415.928759, val_AUC:0.824673, val_pAUC:0.671154
100%|██████████| 20/20 [00:03<00:00,  6.26it/s]
2020-12-01 09:34

100%|██████████| 20/20 [00:03<00:00,  5.73it/s]
2020-12-01 09:49:54,958 - pytorch_modeler.py - INFO - Epoch 292/300:train_loss:337.279461, tr_rec:333.875301, tr_eng:-36.419158, tr_covd:1409.215366, val_AUC:0.806769, val_pAUC:0.629036
100%|██████████| 20/20 [00:03<00:00,  5.66it/s]
2020-12-01 09:50:32,015 - pytorch_modeler.py - INFO - Epoch 293/300:train_loss:337.759632, tr_rec:336.087679, tr_eng:-54.200884, tr_covd:1418.408627, val_AUC:0.760597, val_pAUC:0.513863
100%|██████████| 20/20 [00:02<00:00,  7.49it/s]
2020-12-01 09:51:09,566 - pytorch_modeler.py - INFO - Epoch 294/300:train_loss:337.755233, tr_rec:331.642778, tr_eng:-9.760581, tr_covd:1417.703384, val_AUC:0.787564, val_pAUC:0.528859
100%|██████████| 20/20 [00:03<00:00,  5.91it/s]
2020-12-01 09:51:47,608 - pytorch_modeler.py - INFO - Epoch 295/300:train_loss:336.989951, tr_rec:332.665156, tr_eng:-27.592779, tr_covd:1416.814125, val_AUC:0.801602, val_pAUC:0.580083
100%|██████████| 20/20 [00:03<00:00,  5.82it/s]
2020-12-01 09:52:

elapsed time: 11277.904274702 [sec]
use: cuda:0


100%|██████████| 38/38 [00:15<00:00,  2.49it/s]
2020-12-01 09:55:57,113 - pytorch_modeler.py - INFO - Epoch 1/300:train_loss:96077.855530, tr_rec:95991.009644, tr_eng:628.556458, tr_covd:4798.049500, val_AUC:0.511026, val_pAUC:0.485926
100%|██████████| 38/38 [00:06<00:00,  5.84it/s]
2020-12-01 09:56:49,327 - pytorch_modeler.py - INFO - Epoch 2/300:train_loss:94776.899353, tr_rec:94697.074585, tr_eng:566.394599, tr_covd:4637.082691, val_AUC:0.569183, val_pAUC:0.521141
100%|██████████| 38/38 [00:06<00:00,  5.46it/s]
2020-12-01 09:57:42,730 - pytorch_modeler.py - INFO - Epoch 3/300:train_loss:93707.413940, tr_rec:93645.295532, tr_eng:407.907003, tr_covd:4265.546654, val_AUC:0.519441, val_pAUC:0.518272
100%|██████████| 38/38 [00:06<00:00,  6.19it/s]
2020-12-01 09:58:35,189 - pytorch_modeler.py - INFO - Epoch 4/300:train_loss:92688.826843, tr_rec:92650.968567, tr_eng:184.058114, tr_covd:3890.470051, val_AUC:0.593870, val_pAUC:0.526038
100%|██████████| 38/38 [00:06<00:00,  5.61it/s]
2020-12-

100%|██████████| 38/38 [00:06<00:00,  6.00it/s]
2020-12-01 10:21:52,849 - pytorch_modeler.py - INFO - Epoch 30/300:train_loss:758.662740, tr_rec:758.537011, tr_eng:-109.454193, tr_covd:2214.230185, val_AUC:0.518831, val_pAUC:0.505219
100%|██████████| 38/38 [00:06<00:00,  5.54it/s]
2020-12-01 10:22:48,010 - pytorch_modeler.py - INFO - Epoch 31/300:train_loss:753.359963, tr_rec:754.266327, tr_eng:-116.276442, tr_covd:2144.256306, val_AUC:0.582310, val_pAUC:0.507483
100%|██████████| 38/38 [00:06<00:00,  5.53it/s]
2020-12-01 10:23:41,293 - pytorch_modeler.py - INFO - Epoch 32/300:train_loss:744.771579, tr_rec:745.205837, tr_eng:-110.454203, tr_covd:2122.232880, val_AUC:0.564500, val_pAUC:0.508127
100%|██████████| 38/38 [00:06<00:00,  5.61it/s]
2020-12-01 10:24:34,406 - pytorch_modeler.py - INFO - Epoch 33/300:train_loss:737.512503, tr_rec:738.111087, tr_eng:-115.278124, tr_covd:2185.845343, val_AUC:0.555970, val_pAUC:0.504722
100%|██████████| 38/38 [00:06<00:00,  5.44it/s]
2020-12-01 10:25

history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)